In [59]:
from bs4 import BeautifulSoup
import json
import datetime
import csv
import os

In [68]:
def save_to_csv(data, csv_file):
    file_exists = os.path.isfile(csv_file)

    with open(csv_file, mode='a', encoding='utf-8', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=data.keys())

        if not file_exists:
            writer.writeheader()

        row = data.copy()
        row['requisitos'] = '; '.join(row['requisitos']) if row['requisitos'] else ''
        print(row)
        writer.writerow(row)

In [69]:
def extract_job_details(job_info_html):
    """
    Extrae detalles del trabajo desde el HTML guardado como texto.
    
    :param job_info_html: Contenido HTML como texto
    :type job_info_html: str
    :return: Diccionario con detalles estructurados del trabajo
    :rtype: dict
    """
    result = {
        'titulo': None,
        'empresa': None,
        'ubicacion': None,
        'fecha_publicacion': None,
        'fecha_scrapping': None,
        'descripcion': None,
        'requisitos': None,
    }
    
    # Usar BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(job_info_html, 'html.parser')
    
    # Extraer el título del trabajo
    titulo_element = soup.find('p', class_='title_offer')
    if titulo_element:
        result['titulo'] = titulo_element.text.strip()
    
    # Extraer información del encabezado (empresa y ubicación)
    header_detail = soup.find('div', class_='header_detail')
    if header_detail:
        # Extraer nombre de la empresa
        empresa_element = header_detail.find('p', class_='fwB fs16')
        if empresa_element:
            result['empresa'] = empresa_element.text.strip()
        
        # Extraer ubicación (generalmente es el tercer párrafo en header_detail)
        ubicacion_p = None
        ps = header_detail.find_all('p')
        for p in ps:
            if 'mb5' in p.get('class', []):
                ubicacion_p = p
                break
        
        if ubicacion_p:
            result['ubicacion'] = ubicacion_p.text.strip()

    fecha_element = soup.find('p', class_='fc_aux fs13 mtB')
    fecha_element = soup.select_one('p.fc_aux.fs13.mtB')

    if fecha_element:
        fecha = fecha_element.get_text(strip=True)
        result['fecha_publicacion'] =fecha

    result['fecha_scrapping'] = (datetime.datetime.now() - datetime.timedelta(days=7)).strftime('%Y-%m-%d')

    descripcion_elem = soup.select_one('div.fs16.t_word_wrap')
    descripcion = descripcion_elem.get_text(strip=True) if descripcion_elem else ""
    result['descripcion'] = descripcion

    requisitos_elem = soup.select('ul.fs16.disc.mbB li.mb5')
    requerimientos = [li.get_text(strip=True) for li in requisitos_elem]
    result['requisitos'] = requerimientos

        
    return result

In [70]:
def process_job_details(job_details_json, output_csv):
    """
    Procesa el diccionario de trabajos y extrae la información estructurada.
    
    :param job_details_json: Diccionario con títulos de trabajo como claves y HTML como valores
    :type job_details_json: dict
    :return: Diccionario con información estructurada de los trabajos
    :rtype: dict
    """
    structured_jobs = {}
    
    for job_title, job_html in job_details_json.items():
        job_info = extract_job_details(job_html)
        save_to_csv(job_info, output_csv)

    
    return structured_jobs

In [71]:
if __name__ == "__main__":
    try:
        with open('job_details.json', 'r', encoding='utf-8') as file:
            job_details = json.load(file)
        structured_jobs = process_job_details(job_details, 'job_results.csv')
    except FileNotFoundError:
        print("No se encontró el archivo job_details.json")
    except json.JSONDecodeError:
        print("Error al decodificar el archivo JSON")

{'titulo': 'Convocatoria Masiva / Restaurante Parrillero GYU Magdalena', 'empresa': 'Grupo EDO', 'ubicacion': 'Magdalena Del Mar, Lima', 'fecha_publicacion': 'Hace  44   minutos', 'fecha_scrapping': '2025-04-23', 'descripcion': 'SOMOS EL RESTAURANTE DE CARNES Y PARRILLAS GYU!Requerimos cubrir dentro de nuestro equipo a un Vajillero / Anfitriona / Ayudante de Bar/ Anfitriona / CORREDORES entre otros puestos en Gyu Sede Magdalena.Requisitos:- Disponibilidad para trabajar en el Distrito de Magdalena.- Realizar horarios de rotativos, según la necesidad de la operación.- Experiencia 06 meses mínimo en el puesto.Beneficios:- Ingreso a planilla- Almuerzo- Amplias posibilidades de desarrollo- Beneficios corporativos- BonificacionesConsejo: ¡No vengas solo! Puedes traer a tus AMIGOS a esta oportunidad laboral, porque junto a ellos siempre es mejor.  Cuantos más sean, más divertido y lleno de logros compartidos será el camino. ¡No lo pienses más y vive la Edo experiencia, estamos felicesdetenert

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17232\1038135965.py:21: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(job_info_html, 'html.parser')


{'titulo': '!Chorrillos! Asesor Con o Sin Exp/Planilla Completa/Descansos Fijos Domingos y Feriados/Turno AM', 'empresa': 'solo 5 dias de capacitacion Firmas contrato', 'ubicacion': 'Chorrillos, Lima', 'fecha_publicacion': 'Hace  2  días', 'fecha_scrapping': '2025-04-23', 'descripcion': '¡¡¡BUSCAMOS\xa0A\xa0LOS\xa0MEJORES\xa0ASESORES!!!¡¡¡PORQUE\xa0ESPERAR\xa0MÁS\xa0,\xa0POSTULA\xa0AHORA\xa0Y\xa0GANA\xa0MÁS!!!ASESORES\xa0DE\xa0VENTAS-\xa0PRESENCIAL\xa0SEDE\xa0CHORRILLOS-\xa0GALAXY\xa0PLAZAFUNCIÓN:\xa0Llamar\xa0a\xa0clientes\xa0de\xa0otros\xa0operadores\xa0para\xa0ofrecerles\xa0migrar\xa0a\xa0claro\xa0con\xa0su\xa0RUC\xa010,\xa0RUC\xa015,\xa0RUC\xa020\xa0o\xa0DNI\xa0,\xa0ofreciéndoles\xa0descuentos\xa0y\xa0beneficios\xa0al\xa0unirse\xa0a\xa0la\xa0familia\xa0CLAROREQUISITOS:CON\xa0O\xa0SIN\xa0EXPERIENCIAASISTIR\xa0PRESENCIALMENTE\xa0AL\xa0GALAXY\xa0PLAZA\xa0CHORRILLOS\xa0(CERCA\xa0AL\xa0TOTTUS\xa0DE\xa0HUAYLAS,\xa0CHORRILLOS)HORARIO\xa0DE\xa0CAPACITACIÓN\xa0PRESENCIAL:-\xa05\xa0días\xa0d

KeyboardInterrupt: 